# Imports libraries

In [ ]:
import pandas as pd
import numpy as np
import swifter
from pathlib import Path
import os
import base64
from requests import post, get
import json 

# Dummy 1

## Data loading and cleaning

In [ ]:
def get_absolute_path(relative_path):
    '''
    Creates an absolute path based on a relative path provided
    '''
    try:
        path = str(os.getcwd()).split("/")
        path = "/".join([i for i in path[:-1]])
        absolute_path = path + "/" + relative_path
        return absolute_path
    except Exception as error:
        print(error)
        raise

try:
    ## Loads the dataset
    path = get_absolute_path(r"Data/charts.csv")
    df = pd.read_csv(path)

    ## Drops duplicates
    df.sort_values(by= ["date", "position"], inplace= True)
    df.drop_duplicates(subset= ["name", "position", "date"], keep= "first", inplace= True)

    ## Transform dates into datetime objects
    df.date = pd.to_datetime(df.date)
except Exception as error:
    print(error)
    raise

In [ ]:
try:
    artists = []
    for (index, row) in df.iterrows():
        artist = row.artists.replace("[", "").replace("'", "").replace("]", "")
        artist = artist.split(",")
        artist = artist[0].strip()
        artists.append(artist)

    df.drop(columns= ["artists"], inplace= True)
    df["artists"] = artists
except Exception as error:
    print(error)
    raise

## Dummy

In [ ]:
try:
    variable = []
    songs = {}
    previous_date = {}

    for date in df.date.unique():
        temp_df = df.loc[(df.date == date)]
        for (index, row) in temp_df.iterrows():
            if row.name not in songs:
                variable.append(0)
                songs[row.name] = date
            else:
                if previous_date == songs.get(row.name):
                    variable.append(1)
                    songs[row.name] = date
                elif previous_date != songs.get(row.name):
                    variable.append(0)
                    songs[row.name] = date
        
        previous_date = date

    df["dummy_1"] = variable
except Exception as error:
    print(error)
    raise

# Release Date

In [ ]:
## Defines client credentials for the API and some functions
## used to make the request
client_id = '1435ba72a607475eab2e2184cabc2777'
client_secret = '61951b1287e94f91bb4a57c664606e59'

def get_token():
    '''
    Generates a token to make an API request
    '''
    auth_string= client_id + ':' + client_secret
    auth_bytes = auth_string.encode('utf-8')
    auth_base64 = str(base64.b64encode(auth_bytes), 'utf-8')

    url = 'https://accounts.spotify.com/api/token'
    headers = {"Authorization" : 'Basic '+ auth_base64,
               'Content_type': 'application/x-www-form-urlencoded'
               }
    data = {'grant_type': 'client_credentials'}
    result = post(url, headers = headers, data = data)
    json_result = json.loads(result.content)
    token = json_result['access_token']
    return token

def get_auth_header(token):
    return {'Authorization': 'Bearer ' + token }

def get_features(token, song_id):
    '''
    Retreives all the information based on the track ID passed
    '''
    url = f'https://api.spotify.com/v1/tracks/{song_id}?market=US'
    headers = get_auth_header(token)
    result = get(url , headers = headers)
    json_result = json.loads(result.content)

    return json_result

token = get_token()

In [ ]:
data = {"track_id": [], "release_date": []}
for id in df.track_id.unique():
    track = get_features(token, id)
    data["track_id"].append(id)
    data["release_date"].append(track["album"]["release_date"])

dates = pd.DataFrame(data= data)